In [5]:
import json
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow_hub as hub
# from ckiptagger import data_utils, construct_dictionary, WS, POS, NER
# from huggingface_hub import hf_hub_download
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold

In [6]:
print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

Version:  2.13.0
Eager mode:  True
Hub version:  0.16.1
GPU is NOT AVAILABLE


In [9]:
# 讀取.json
with open('./comments.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# 提取 'comment' 與 'ad'
comments = [entry['comment'] for entry in data]
ads = [entry['ad'] for entry in data]

In [10]:
embed_url = "https://tfhub.dev/google/nnlm-zh-dim50/2"
hub_layer = hub.KerasLayer(embed_url, input_shape=[], dtype=tf.string, trainable=True)

In [11]:
# 將資料分成訓練集和測試集（4:1）
X_train, X_test, y_train, y_test = train_test_split(comments, ads, test_size=0.2, random_state=42, stratify=ads)

In [13]:
# 定義交叉驗證的折數
num_folds = 5
# 初始化 StratifiedKFold
kfold = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)

In [14]:
# 創建空列表來保存每個折數的訓練和驗證分數
train_scores = []
val_scores = []

In [16]:
for fold, (train_index, val_index) in enumerate(kfold.split(X_train, y_train)):
    print(f"Training on fold {fold + 1}/{num_folds}...")
    # 根據索引拆分訓練和驗證資料
    trainX = np.array([X_train[i] for i in train_index])
    valX = np.array([X_train[i] for i in val_index])
    trainY = np.array([y_train[i] for i in train_index])
    valY = np.array([y_train[i] for i in val_index])

    model = tf.keras.Sequential([
        hub_layer,
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(1)
    ])

    # 編譯模型
    model.compile(optimizer='adam',
                  loss=tf.losses.BinaryCrossentropy(from_logits=True),
                  metrics=[tf.metrics.BinaryAccuracy(threshold=0.0, name='accuracy')])

    # 訓練模型
    history = model.fit(trainX,
                        trainY,
                        epochs=40,
                        batch_size=512,
                        verbose=0)

    # 評估模型
    train_score = model.evaluate(trainX, trainY, verbose=0)
    val_score = model.evaluate(valX, valY, verbose=0)

    train_scores.append(train_score)
    val_scores.append(val_score)

Training on fold 1/5...
Training on fold 2/5...
Training on fold 3/5...
Training on fold 4/5...
Training on fold 5/5...


In [17]:
# 輸出每個折數的平均訓練和驗證分數
print("Average training scores:", np.mean(train_scores, axis=0))
print("Average validation scores:", np.mean(val_scores, axis=0))

Average training scores: [0.01280002 0.99293369]
Average validation scores: [0.17621403 0.96918956]


In [23]:
y_pred = model.predict(X_test, verbose=1)
y_pred = (y_pred > 0.5).astype(np.integer)

59/59 [==============================] - 3s 46ms/step


C:\Users\501-Tsai\AppData\Local\Temp\ipykernel_9572\2983523565.py:2: DeprecationWarning: Converting `np.integer` or `np.signedinteger` to a dtype is deprecated. The current result is `np.dtype(np.int_)` which is not strictly correct. Note that the result depends on the system. To ensure stable results use may want to use `np.int64` or `np.int32`.
  y_pred = (y_pred > 0.5).astype(np.integer)


In [63]:
y_test_np = np.array(y_test)
y_test_np = y_test_np[:, np.newaxis]
accuracy = (y_pred == y_test_np).mean()
ad00 = np.logical_and(y_test_np == 0, y_pred == 0)
ad01 = np.logical_and(y_test_np == 0, y_pred == 1)
ad10 = np.logical_and(y_test_np == 1, y_pred == 0)
ad11 = np.logical_and(y_test_np == 1, y_pred == 1)
print("測試資料的數量:", len(y_test))
print("準確度:", accuracy)
print("本身不是廣告且判斷不是廣告的數量:", sum(ad00))
print("本身不是廣告且判斷是廣告的數量:", sum(ad01))
print("本身是廣告且判斷不是廣告的數量:", sum(ad10))
print("本身是廣告且判斷是廣告的數量:", sum(ad11))


測試資料的數量: 1867
準確度: 0.965720407070166
本身不是廣告且判斷不是廣告的數量: [1786]
本身不是廣告且判斷是廣告的數量: [30]
本身是廣告且判斷不是廣告的數量: [34]
本身是廣告且判斷是廣告的數量: [17]


In [50]:
X_test_np = np.array(X_test)
X_test_np = X_test_np[:, np.newaxis]

In [53]:
ad_pred = X_test_np[y_pred == 1]
print("判斷為廣告的數量:", len(ad_pred))
print("判斷為廣告的句子:\n", ad_pred)

判斷為廣告的數量: 47
判斷為廣告的句子:
 ['我已經連續9天都無法創腳色玩，全部都是創角受限，搞屁' '白痴喔，根本不讓人認證伺服器是怎樣'
 '可以改回來50等能刪角色或40等因為試玩一種角色大約都這等級就知自己想不想玩這角色如果不想玩想玩別的角色又不能刪覺得很奇怪可以改嗎'
 '爛了啦破遊戲'
 '聽覺上的享受!以後再也不會無聊。還有壹個特別的小技巧告訴大家。在EK433.?om 這裏下載的版本。可以滿V。不用自己充。終於收到我需要的寶貝了。東西很好。價美物廉。說實在這是我來讓我最滿意的一次購物。無論是掌櫃的態度還是對物品。我都非常滿意的。掌櫃態度很專業熱情。有問必答。回復也很快。我問了不少問題。他都不覺得煩。'
 '聽覺上的享受！以後再也不會無聊。還有壹個特別的小技巧告訴大家。在2021MG。?OM 這裏下載的版本。可以滿V。不用自己充。終於收到我需要的寶貝了。東西很好。價美物廉。說實在這是我來讓我最滿意的一次購物。無論是掌櫃的態度還是對物品。我都非常滿意的。掌櫃態度很專業熱情。有問必答。回復也很快。我問了不少問題。他都不覺得煩。'
 '黑橘的陰謀 1 沒葉子(花錢) 爛遊戲大家別玩'
 '視覺上的享受！以後再也不會無聊。還有壹個特別的小技巧告訴大家。在ＴＷＤ122.?OM 這裏下載的版本。可以滿V。不用自己充。終於收到我需要的寶貝了。東西很好。價美物廉。說實在這是我來讓我最滿意的一次購物。無論是掌櫃的態度還是對物品。我都非常滿意的。掌櫃態度很專業熱情。有問必答。回復也很快。我問了不少問題。他都不覺得煩。'
 '精神上的享受！以後再也不會無聊。還有壹個特別的小技巧告訴大家。在Swa33，希OM 這裏下載的版本。可以滿V。不用自己充。終於收到我需要的寶貝了。東西很好。價美物廉。說實在這是我來讓我最滿意的一次購物。無論是掌櫃的態度還是對物品。我都非常滿意的。掌櫃態度很專業熱情。有問必答。回復也很快。我問了不少問題。他都不覺得煩。'
 '排隊等了一小時還沒進入   打怪幾百搶一隻怪...一個遊戲可以廣告打這麼大，分數卻是這麼低也厲害了' '好玩，繼續努力' '垃圾遊戲！！！'
 '千萬不要玩天堂m，這遊戲不要入坑，遊戲公司態度惡劣，機率號稱10%以上，可是實際機率不到1%分，變身與娃娃卡片不論抽卡與合成都比公告還要低無數倍，更別說玩家還會被遊戲公司告，千萬別入

In [55]:
ad_test = X_test_np[np.logical_and(y_pred == 0, y_test_np == 1)]
print("應為廣告卻被誤判的數量:", len(ad_test))
print("應為廣告卻被誤判的句子:\n", ad_test)

應為廣告卻被誤判的數量: 34
應為廣告卻被誤判的句子:
 ['讓我告訴你HBO277點c歐m上可以體驗一下無縣圓寶手遊，登入就享至尊會員，還有原創角色交易系統，玩膩了可以換現金。但自從最近更新了就不能進入遊戲。一開遊戲就閃退，帳號也一直不能成功梆定。希望有人可連絡我提供協助。人物和景色處理都獨樹一格，每-個地方都環環相扣，讓我體驗到遊戲設計師的創意小巧思，非常耐玩，而且劇情引人入勝，但是在敘事的時間也不會冗長，把握的剛剛好，跟我以前玩的手遊完全不一樣，是一個有層次跟故事的遊戲。'
 '酣暢淋漓的華麗特技,更重要的是推薦去找蒼天牧師網玩鑽石無限滿V版,各種資源全都能無限這樣玩法太牛了,還有幾千個爆款無限版遊戲,前陣子好像有緊急維修,但事後沒有什麼補償給玩家感受上不是很好,但看在平時掉落那麼多獎勵的份上還是沒什麼特別不開心了,不過我還是建議下一次要補償一下,這樣才能夠更加留住玩家啊!這遊戲這麼好玩得讓更多人知道才行.'
 "聽覺上的享受'以後再也不會無聊。還有壹個特別的小技巧告訴大家。在網址ＨＣCD9.?OM可以學習如何創造以太幣的睡後收入.躺著就能賺到幣.我今天已經收了三w多超爽.後面甭看了.但不知道是不是之前有更新過.更新了目前最新版本以後.比上一個版本得還更容易卡住.然後就要用縮小畫面來重新更新.一個不小心就要重新啟動。他都不觉得烦"
 '超棒的說 活動填滿了小說和動漫的坑 ，我是在knn83.c om找手遊，裡面無需儲值就可以滿級會員和無限錢幣，進去就先送你神裝神獸，每週都還有很多活動禮包更新。還特地再做了動畫 遊戲體驗嗎 感覺出貨很容易的 至少比FGO好了 打的時候也很順手的 就是慢了點 如果週回速度可以再提高的話就更好 活動關卡有掉落物顯示就好 比起SAOMD 好很多 砸了不少錢下去 總括而言 真的挺好的 需要進步的位也有的 佢望能像FGO一樣 一路進步'
 '美好的清晨來杯咖啡再配上破解手遊，你會發現生活就是如此簡單，先介紹9MAQ。?óＭ，上線滿V級版可以玩，資源元寶都是無限用的，上線就送無限貨幣，有點晚才發現。不管是場上的判斷還是選角色的判斷多都變化多端，雖然有時會遇到雷對友，但只要和能信任的朋友玩就還好，還有角色很好抽，又可以和麻吉朋友一起玩，這款遊戲讓我今年每天都很快樂。'
 '哇塞～終於出了，遊戲等你來體驗，大家可以去SSr25.C〇м裡面許多